In [1]:
import sys
import os

# Get the current working directory (where the notebook is located)
current_dir = os.getcwd()

# Add the parent directory to sys.path
sys.path.append('../models')
sys.path.append('../scripts')

In [2]:
import torch
import torch.nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image
from torch import nn
import torch.nn.functional as F
import os
from random import randint
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from autoencoder import ResNetAutoencoder
from dae import DenoisingAutoencoder
from mtg_cards_dataset import MTGCardsDataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

BATCH_SIZE = 20
NUM_EPOCHS = 1
LOGGING_INTERVAL = 150
SAVE_MODEL = True
LOAD_MODEL = True
DATA_DIR = "../data"

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),          # Convert images to tensors
    transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1] 
])

dataset = MTGCardsDataset(DATA_DIR, transform=transform)
dataloader = DataLoader(dataset, batch_size=1)

In [ ]:
autoencoder = ResNetAutoencoder()
autoencoder.to(device)
autoencoder.load_state_dict(torch.load('trained_models/autoencoder.pth'))

dae = DenoisingAutoencoder()
dae.to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(dae.parameters(), lr=1e-3)

if LOAD_MODEL:
    dae.load_state_dict(torch.load('trained_models/dae.pth'))
    
# Training Loop
num_epochs = 2
dae.train()  # Set the DAE to training mode

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i, (clean_images, _) in enumerate(dataloader):  # Load only clean images from the dataset
        clean_images = clean_images.to(device)
        
        # Generate noisy images using the noisy image generator
        with torch.no_grad():  # No need to calculate gradients for the noisy generator
            noisy_images = autoencoder(clean_images)
        
        # Forward Pass through DAE
        denoised_images = dae(noisy_images)
        
        # Calculate Loss
        loss = loss_fn(denoised_images, clean_images)  # Compare with the original clean images
        
        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

        if i % LOGGING_INTERVAL == 0:
            print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item()}")
            idx = randint(0, len(dataset))
            img, _ = dataset[idx]
            dae.test(img)

            # Save model if required
            if SAVE_MODEL:
                print('SAVING MODEL. DO NOT INTERRUPT!')
                torch.save(dae.state_dict(), 'trained_models/dae.pth')
                print('MODEL SAVED!')
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(dataloader):.4f}")

C:\Users\agill\AppData\Local\Temp\ipykernel_3520\3113659833.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  autoencoder.load_state_dict(torch.load('trained_models/autoen

PermissionError: [Errno 13] Permission denied: 'trained_models'